### Look up the Adam optimization functions in PyTorch https://pytorch.org/docs/stable/optim.html . How does it work? Try at least one other optimization function with the diabetes dataset shown in class. How does the model perform with the new optimizer? Did it perform better or worse than Adam? Why do you think that is?

In [1]:
import pandas as pd
import torch

diabetes_df = pd.read_csv("../week13repo/diabetes.csv")
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [12]:
import torch.nn as nn
import torch.nn.functional as F #this has activation functions

# Creating tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[-0.8514, -0.9801, -0.4048,  ..., -0.6077,  0.3108, -0.7922],
        [ 0.3566,  0.1614,  0.4654,  ..., -0.3021, -0.1164,  0.5610],
        [-0.5494, -0.5045, -0.6223,  ...,  0.3726, -0.7649, -0.7076],
        ...,
        [-0.8514, -0.7582,  0.0303,  ...,  0.7800, -0.7861, -0.2847],
        [ 1.8665, -0.3142,  0.0303,  ..., -0.5695, -1.0194,  0.5610],
        [ 0.0546,  0.7322, -0.6223,  ..., -0.3149, -0.5770,  0.3073]])


In [13]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=8, hidden1=20, hidden2=20, out_features =2):
        super().__init__()
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [14]:
torch.manual_seed(42)

#instantiate the model
model = ANN_Model()

In [17]:
# loss function USING SGD Algorithm
loss_function = nn.CrossEntropyLoss()

#optimizer
#optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)


In [18]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss.item()}')
    
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() #for backward propagation 
    optimizer.step() #performs one optimization step each epoch
    

Epoch number: 1 with loss: 0.6595041155815125
Epoch number: 11 with loss: 0.6132243871688843
Epoch number: 21 with loss: 0.5253595113754272
Epoch number: 31 with loss: 0.47467175126075745
Epoch number: 41 with loss: 0.4445890188217163
Epoch number: 51 with loss: 0.433291494846344
Epoch number: 61 with loss: 0.42537766695022583
Epoch number: 71 with loss: 0.41854599118232727
Epoch number: 81 with loss: 0.41132161021232605
Epoch number: 91 with loss: 0.4042525291442871
Epoch number: 101 with loss: 0.3967641294002533
Epoch number: 111 with loss: 0.3894926905632019
Epoch number: 121 with loss: 0.3818753957748413
Epoch number: 131 with loss: 0.3738797605037689
Epoch number: 141 with loss: 0.36500146985054016
Epoch number: 151 with loss: 0.3556433916091919
Epoch number: 161 with loss: 0.3455562889575958
Epoch number: 171 with loss: 0.3349319100379944
Epoch number: 181 with loss: 0.32496872544288635
Epoch number: 191 with loss: 0.3151437044143677
Epoch number: 201 with loss: 0.304991573095321

In [19]:
#predictions
y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = model(data)
        y_pred.append(prediction.argmax().item())


In [20]:
from sklearn.metrics import accuracy_score
a_score = accuracy_score(y_test, y_pred)
print(a_score)

0.6363636363636364


In [21]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.61      0.69       100
           1       0.49      0.69      0.57        54

    accuracy                           0.64       154
   macro avg       0.63      0.65      0.63       154
weighted avg       0.68      0.64      0.64       154



In [ ]:
USING SGD Algorithm accuracy is .64%

trying to figure out accuracy using Adam Algorithm

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [23]:
import torch.nn as nn
import torch.nn.functional as F #this has activation functions

# Creating tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[-0.8514, -0.9801, -0.4048,  ..., -0.6077,  0.3108, -0.7922],
        [ 0.3566,  0.1614,  0.4654,  ..., -0.3021, -0.1164,  0.5610],
        [-0.5494, -0.5045, -0.6223,  ...,  0.3726, -0.7649, -0.7076],
        ...,
        [-0.8514, -0.7582,  0.0303,  ...,  0.7800, -0.7861, -0.2847],
        [ 1.8665, -0.3142,  0.0303,  ..., -0.5695, -1.0194,  0.5610],
        [ 0.0546,  0.7322, -0.6223,  ..., -0.3149, -0.5770,  0.3073]])


In [24]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=8, hidden1=20, hidden2=20, out_features =2):
        super().__init__()
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [25]:
torch.manual_seed(42)

#instantiate the model
model = ANN_Model()

In [27]:
# loss function
loss_function = nn.CrossEntropyLoss()

#optimizer
#optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)


In [28]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss.item()}')
    
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() #for backward propagation 
    optimizer.step() #performs one optimization step each epoch
    

Epoch number: 1 with loss: 0.6151040196418762
Epoch number: 11 with loss: 0.4274582266807556
Epoch number: 21 with loss: 0.36235126852989197
Epoch number: 31 with loss: 0.3042708933353424
Epoch number: 41 with loss: 0.2522340416908264
Epoch number: 51 with loss: 0.21815843880176544
Epoch number: 61 with loss: 0.1969689428806305
Epoch number: 71 with loss: 0.16857154667377472
Epoch number: 81 with loss: 0.15262673795223236
Epoch number: 91 with loss: 0.141379252076149
Epoch number: 101 with loss: 0.1857585608959198
Epoch number: 111 with loss: 0.16066332161426544
Epoch number: 121 with loss: 0.1275348961353302
Epoch number: 131 with loss: 0.1123829111456871
Epoch number: 141 with loss: 0.09853900969028473
Epoch number: 151 with loss: 0.08790972083806992
Epoch number: 161 with loss: 0.08027870208024979
Epoch number: 171 with loss: 0.07695543766021729
Epoch number: 181 with loss: 0.07021258026361465
Epoch number: 191 with loss: 0.06659486889839172
Epoch number: 201 with loss: 0.0662702918

In [29]:
#predictions
y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = model(data)
        y_pred.append(prediction.argmax().item())


In [30]:
from sklearn.metrics import accuracy_score
a_score = accuracy_score(y_test, y_pred)
print(a_score)

0.6818181818181818


In [31]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.74      0.75       100
           1       0.54      0.57      0.56        54

    accuracy                           0.68       154
   macro avg       0.65      0.66      0.65       154
weighted avg       0.69      0.68      0.68       154



In [ ]:
USING SGD Algorithm accuracy is .68%

### Write a function that lists and counts the number of divisors for an input value. 


Example 1: 
    
Input: 5 
        
Output: “There are 2 divisors: 1 and 5

In [10]:
def divisor(x):
   print("The factors of",x,"are:")
   for i in range(1, x + 1):
       if x % i == 0:
           print(i)

num = 40

divisor(num)


The factors of 40 are:
1
2
4
5
8
10
20
40
